Below is TOPSIS method


In [502]:
import numpy as np
import pandas as pd
a = np.array([[1,2,3],[4,5,6]])
a.sum(axis = 0)

#from to_latex_table import array_latex_table, csv_latex_table


array([5, 7, 9])

In [503]:
import numpy as np
import pandas as pd

def array_latex_table(mat):
    """
    input: np.array
    output:String
    """
    if len(mat.shape) != 2:
        raise ValueError("Input matrix must be 2-dimensional.")
    rows, cols = mat.shape
    latex_str = ""
    for i in range(rows):
        row_str = " & ".join([str(mat[i, j]) for j in range(cols)])
        latex_str += row_str + " \\\\"
    return latex_str

def csv_latex_table(file_path):
    """
    input: csv file path
    output:String
    """
    df = pd.read_csv(file_path)
    latex_str = df.to_latex(index=False, header=True)
    return latex_str

array_latex_table(np.array([[0.05  ,     0.04225352 ,0.07416564 ,0.02912621 ,0.02325581],
 [0.25   ,    0.21126761 ,0.17305315 ,0.34951456 ,0.34883721],
 [0.35 ,      0.63380282 ,0.51915946 ,0.46601942 ,0.34883721],
 [0.2   ,     0.07042254 ,0.12978986 ,0.11650485 ,0.20930233],
 [0.15   ,    0.04225352 ,0.10383189 ,0.03883495 ,0.06976744]]).T)

'0.05 & 0.25 & 0.35 & 0.2 & 0.15 \\\\0.04225352 & 0.21126761 & 0.63380282 & 0.07042254 & 0.04225352 \\\\0.07416564 & 0.17305315 & 0.51915946 & 0.12978986 & 0.10383189 \\\\0.02912621 & 0.34951456 & 0.46601942 & 0.11650485 & 0.03883495 \\\\0.02325581 & 0.34883721 & 0.34883721 & 0.20930233 & 0.06976744 \\\\'

**==========evaluation methods================**

In [ ]:
def topsis(matrix, weights, criteria_types):
    """
    Perform TOPSIS ranking.
    
    Parameters
    ----------
    matrix : 2D array-like
        Decision matrix with shape (m, n) — m alternatives, n criteria.
    weights : 1D array-like
        Importance weights for each criterion (should sum to 1).
    criteria_types : list of str
        Each element is 'benefit' or 'cost' indicating the type of criterion.
    
    Returns
    ----------
    scores : ndarray
        Closeness coefficient for each alternative (higher = better).
    rankings : ndarray
        Indices of alternatives ranked from best to worst.
    """
    # Avoid wrong imports
    
    if (sum(weights) - 1) > 0.001:
        raise ValueError("Weights must sum to 1.")
    
    # Step 1: Normalize the decision matrix
    print(matrix)
    norm_matrix = matrix / np.sqrt((matrix ** 2).sum())
    print(f"norm_matrix = {norm_matrix}")
    
    # Step 2: Multiply by weights
    weighted_matrix = norm_matrix * weights

    # Step 3: Determine ideal and negative-ideal solutions
    ideal_solution = []
    negative_ideal_solution = []
    for i in range(matrix.shape[1]):
        if criteria_types[i] == '+':
            ideal_solution.append(weighted_matrix[:, i].max())
            negative_ideal_solution.append(weighted_matrix[:, i].min())
        else:
            ideal_solution.append(weighted_matrix[:, i].min())
            negative_ideal_solution.append(weighted_matrix[:, i].max())
    
    ideal_solution = np.array(ideal_solution)
    negative_ideal_solution = np.array(negative_ideal_solution)

    # Step 4: Calculate distances to ideal and negative-ideal solutions
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matrix - negative_ideal_solution) ** 2).sum(axis=1))

    # Step 5: Calculate performance score
    performance_score = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

    return performance_score

In [505]:
# from asyncio import log

import numpy as np

# Define the three indicators properly
def entropy(infor):
    infor = np.array(infor)
    return np.sum(-infor * np.log(infor + np.finfo(float).eps)) / np.log(len(infor))

def cost_indicator(matrix):
    matrix = np.array(matrix)
    maxi = np.max(matrix, axis=0)
    matrix = (maxi - matrix) / (np.sum(maxi - matrix, axis=0) + np.finfo(float).eps)
    return matrix

def benefit_indicator(matrix):
    matrix = np.array(matrix)
    return matrix / (np.sum(matrix, axis=0) + np.finfo(float).eps)

def optimal_value_indicator(matrix, optimal_value=None):
    matrix = np.array(matrix)
    if optimal_value is None:
        optimal_value = np.median(matrix, axis=0)
    maxn = np.max(abs(matrix - optimal_value), axis=0)
    maxn[maxn==0] = np.finfo(float).eps  # avoid division by zero
    matrix = 1 - (abs(matrix - optimal_value) / maxn)
    matrix = matrix / (np.sum(matrix, axis=0) + np.finfo(float).eps)
    return matrix

def entropy_weights(matrix, indicators, optimal_value=None):
    matrix = np.array(matrix, dtype=float)
    
    # If indicators is a single function
    import types
    if isinstance(indicators, types.FunctionType):
        if indicators is optimal_value_indicator:
            matrix = indicators(matrix, optimal_value)
        else:
            matrix = indicators(matrix)
    else:
        # indicators can be a list with different indicator per criterion
        n_criteria = matrix.shape[1]
        new_matrix = np.zeros_like(matrix)
        for j in range(n_criteria):
            ind = indicators[j]
            if ind is optimal_value_indicator:
                new_matrix[:, j] = ind(matrix[:, j].reshape(-1,1), optimal_value).flatten()
            else:
                new_matrix[:, j] = ind(matrix[:, j].reshape(-1,1)).flatten()
        matrix = new_matrix

    matrix = np.where(matrix==0, np.finfo(float).eps, matrix)

    # Compute entropy
    entropy_values = np.array([entropy(matrix[:,j]) for j in range(matrix.shape[1])])
    diver = 1 - entropy_values
    weights = diver / np.sum(diver)
    return weights

# Input matrix
matrix = np.array([
    [10, 200, 0.5],
    [20, 300, 0.7],
    [30, 250, 0.9]
])

# Example: different indicator per criterion
indicators = [benefit_indicator, optimal_value_indicator, cost_indicator]

weights = entropy_weights(matrix, indicators)
weights

array([0.05292011, 0.66666667, 0.28041322])

In [506]:
RI_table = {1:0.00, 2:0.00, 3:0.58, 4:0.90, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45, 10:1.49}
def Normalize(A):
    """
    Normalize the pairwise comparison matrix A.
    """
    if type(A)!= np.array:
        A = np.array(A)
    A = np.array(A, dtype=float)
    sj = np.sum(np.abs(A), axis=0)
    A = A / sj
    return A
def AHP(A):
    """
    Compute AHP weights using the eigenvalue method.
    
    Parameters:
        A (np.ndarray): Pairwise comparison matrix (n x n)
    
    Returns:
        weights (np.ndarray): Normalized eigenvector (priority vector)
        lambda_max (float): Maximum eigenvalue
        CI (float): Consistency Index
        CR (float): Consistency Ratio
    """
    if type(A)!= np.array:
        A = np.array(A)

    # --- Step 1: Eigen decomposition ---
    eigvals, eigvecs = np.linalg.eig(A)
    
    # Principal eigenvalue (real part)
    max_index = np.argmax(eigvals.real)
    lambda_max = eigvals[max_index].real
    
    # Principal eigenvector (real part)
    w = eigvecs[:, max_index].real
    
    # Normalize weights
    weights = w / np.sum(w)

    # --- Step 2: Consistency calculations ---
    n = A.shape[0]
    CI = (lambda_max - n) / (n - 1)


    RI = RI_table.get(n, 1.49)  # fallback for n>10

    CR = CI / RI if RI != 0 else 0

    return weights, lambda_max, CI, CR

In [507]:
AHP([[1, 3, 5],
    [1/3, 1, 2],
    [1/5, 1/2, 1]])

(array([0.64832901, 0.22965079, 0.12202019]),
 3.003694598063639,
 0.0018472990318194604,
 0.003184998330723208)

In [508]:
# indicator = [Energy consumption, emission,waste management,water consumption, temperature]
#energy_consumption_weights = [renewable energy ratio, energy efficiency, carbon emissions,renewable energy potential]
energy_consumption_weights = [[1., 36e6/73208, 343, 2744/365, 7.518]]
carbon_emission_of_trans = [[]]
waste_management = []
#water_usage = [Santa Clara, Atlanta  Georgia,  Inglewood CA ,Glendale,Las Vegas-NV,New Orleans, LA,Seatt,Nashville, Denever] 
water_usage = np.array([(80+110)*3.78541178/2,149,None,280,830,(300+380)/2,190,(80+110)*3.78541178/2,125] ) # gallons per person per day
climate = []



In [509]:
def remove_unit(strin,unit):
    return strin.replace(unit,"")
    

**reading data from the dictionary**

In [510]:
#import data
import os
from pathlib import Path
import pandas as pd
def read_csv_from(dir_or_url, filename):
    if isinstance(dir_or_url, Path):
        return pd.read_csv(dir_or_url / filename)
    else:
        # ensure no double slashes
        return pd.read_csv(f"{dir_or_url.rstrip('/')}/{filename}")
#local_data_dir = Path('/Users/daniel/Documents/Math_/Superbowl_vs_environment/data')


    

local_data_dir = Path('/Users/daniel/Documents/Math_/Superbowl_vs_environment/data')

# Raw base that points to the CSV files on GitHub (raw.githubusercontent)
raw_base = "https://raw.githubusercontent.com/maop0/Super-Bowl-Mathematical-Modelling/2139377798b53564aa659dc1185cfba3e5dd84c2/data"


data_dir = local_data_dir if local_data_dir.exists() else raw_base

energy_consumption_data = read_csv_from(data_dir, 'Energy_consmuption.csv')
waste_management_data = read_csv_from(data_dir, 'waste_management.csv')
climate_data = read_csv_from(data_dir, 'climate.csv')
emmision_data = read_csv_from(data_dir, 'transportation.csv')


**==========below we are processing energy_consumption_data==========**

In [511]:
#process energy consumption data

for i in energy_consumption_data.columns:
    
    #removing [int]
    energy_consumption_data[i]= pd.DataFrame(map(lambda x:x.split("[")[0]if type(x) == str else x, energy_consumption_data[i]))
#energy_consumption_data
energy_consumption_data['可再生能源占比 (%)'] = pd.DataFrame(map(lambda x:remove_unit(x.split('[')[0],"%")if type(x) == str else x, energy_consumption_data['可再生能源占比 (%)']))
energy_consumption_data["可再生能源占比 (%)"][0] = energy_consumption_data["可再生能源占比 (%)"][0][1:] 
energy_consumption_data["可再生能源占比 (%)"][1] = energy_consumption_data["可再生能源占比 (%)"][1][1:] 
energy_consumption_data['可再生能源占比 (%)'] = pd.DataFrame(map(lambda x:float(x)/100, energy_consumption_data['可再生能源占比 (%)']))
for i in energy_consumption_data.columns[2:-2]:
    energy_consumption_data[i] = pd.DataFrame(map(float, energy_consumption_data[i]))
energy_consumption_data['平均风速 (km/h)'] = pd.DataFrame(map(lambda x:float(x.split('~')[1]) if x[0] == '~' else float(x), energy_consumption_data['平均风速 (km/h)']))

energy_consumption_data

,城市 (场馆),可再生能源占比 (%),场馆能效 (kWh/座位),电网碳强度 (kgCO₂/kWh),平均太阳辐照 (kWh/m²·日),平均风速 (km/h),说明（数据来源/估算）
0,Santa Clara（Levi’s Stadium）,1.000,0.530,0.184,5.91,8.1,场馆1150块太阳能板年发电可覆盖主场比赛用电
1,Atlanta（Mercedes-Benz Stadium）,1.000,0.507,0.305,5.29,9.4,场馆4000块太阳能板年发1.6GWh
2,Inglewood（SoFi Stadium）,0.570,0.513,0.184,6.21,11.0,场馆无自发电设施，采用加州电网平均可再生比例（约57%）
3,Glendale（State Farm Stadium）,0.140,0.568,0.288,6.59,10.9,采用亚利桑那州平均可再生比例（约14%）
4,Las Vegas（Allegiant Stadium）,1.000,0.554,0.286,6.51,12.7,场馆宣称使用“100%内华达州可再生电力”
5,New Orleans（Superdome）,0.030,0.492,0.420,5.38,15.2,采用路易斯安那州平均可再生比例（≈3%）
6,Seattle（Lumen Field）,0.757,0.524,0.113,4.12,6.6,采用华盛顿州平均可再生比例（约75.7%）
7,Denver（Empower Field）,0.430,0.473,0.358,5.93,13.2,采用科罗拉多州2024年可再生份额（43%）
8,Nashville（Nissan Stadium）,0.130,0.521,0.365,4.96,11.4,采用田纳西州平均可再生比例（≈13%）


**==========below we are processing emission data==========**

In [512]:
#process emission data
emmision_data['公交站数量'] = pd.DataFrame(map(float, emmision_data['公交站数量']))
emmision_data['电车/地铁站数量'] = pd.DataFrame(map(float, emmision_data['电车/地铁站数量']))
emission_data = pd.DataFrame(emmision_data[emmision_data.columns[:-1]])
#emission_data_santa = emission_data(np.where(emission_data['城市']=='Santa Clara, CA	'))
#emission_data_santa
emission_data

,超级碗,年份,城市,场馆,公交站数量,电车/地铁站数量
0,Super Bowl L,2016.0,"Santa Clara, CA",Levi’s Stadium,5.0,3.0
1,Super Bowl LIII,2019.0,"Atlanta, GA",Mercedes-Benz Stadium,3.0,2.0
2,Super Bowl LVI,2022.0,"Inglewood, CA",SoFi Stadium,5.0,2.0
3,Super Bowl LVII,2023.0,"Glendale, AZ",State Farm Stadium,3.0,1.0
4,Super Bowl LVIII,2024.0,"Las Vegas, NV",Allegiant Stadium,2.0,0.0
5,Super Bowl LIX,2025.0,"New Orleans, LA",Caesars Superdome,6.0,1.0
6,Super Bowl LX,2026.0,"Santa Clara, CA",Levi’s Stadium,5.0,3.0
7,Super Bowl LXI,2027.0,"Inglewood, CA",SoFi Stadium,5.0,2.0
8,Super Bowl LXII,2028.0,"Atlanta, GA",Mercedes-Benz Stadium,3.0,2.0
9,Super Bowl LXIII,2029.0,To be determined,NaN,NaN,NaN


**==========below we are processing waste management data==========**

In [513]:
#process waste management data
waste_management_data['总废弃物量'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "t")) if type(x) == str else x, waste_management_data['总废弃物量']))
waste_management_data['总观众人数'] = pd.DataFrame(map(float, waste_management_data['总观众人数']))
waste_management_data['分类回收率'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "%"))/100 if type(x) == str else x, waste_management_data['分类回收率']))
waste_management_data['人均废弃物产生量 (kg/人)'] = waste_management_data['总废弃物量']*1000 / waste_management_data['总观众人数']
waste_santa = np.array([0.75,	2.066667])
waste_Atlanta = np.array([0.90	,1.126761])
waste_Genedale = np.array([0.92,	1.109589])
waste_LV = np.array([0.90	,1.184615])
waste_New_Orleans = np.array([0.91	,1.188525])
waste_management_data

,超级碗,年份,城市,总废弃物量,总观众人数,分类回收率,人均废弃物产生量 (kg/人)
0,Super Bowl L,2016,"Santa Clara, CA",155.0,75000.0,0.75,2.066667
1,Super Bowl LIII,2019,"Atlanta, GA",80.0,71000.0,0.90,1.126761
2,Super Bowl LVI,2022,"Inglewood, CA",70.0,70000.0,0.90,1.000000
3,Super Bowl LVII,2023,"Glendale, AZ",81.0,73000.0,0.92,1.109589
4,Super Bowl LVIII,2024,"Las Vegas, NV",77.0,65000.0,0.90,1.184615
5,Super Bowl LIX,2025,"New Orleans, LA",87.0,73200.0,0.91,1.188525
6,Super Bowl LX,2026,"Santa Clara, CA",NaN,75000.0,NaN,NaN
7,Super Bowl LXI,2027,"Inglewood, CA",NaN,70000.0,NaN,NaN
8,Super Bowl LXII,2028,"Atlanta, GA",NaN,71000.0,NaN,NaN
9,Super Bowl LXIII,2029,To be determined,NaN,NaN,NaN,NaN


**==========below we are processing waste climate data==========**

In [514]:
#process climate data
climate_data["平均每㎡入射太阳能"] = pd.DataFrame(map(lambda x: float(remove_unit(x, "kWh")) if type(x) == str else x, climate_data["平均每㎡入射太阳能"]))
climate_data["平均风速"] = pd.DataFrame(map(lambda x: float(remove_unit(x, " km/h")) if type(x) == str else x, climate_data["平均风速"]))
climate_data['平均温度'] = pd.DataFrame(map(lambda x: float(remove_unit(x, "℃") )if type(x) == str else x, climate_data["平均温度"]))

climate_data

,超级碗,年份,城市,平均每㎡入射太阳能,平均风速,平均温度
0,Super Bowl L,2016,"Santa Clara, CA",3.4,11.2,12.0
1,Super Bowl LIII,2019,"Atlanta, GA",3.7,11.4,8.0
2,Super Bowl LVI,2022,"Inglewood, CA",4.2,12.8,14.0
3,Super Bowl LVII,2023,"Glendale, AZ",4.4,11.1,14.0
4,Super Bowl LVIII,2024,"Las Vegas, NV",4.2,12.6,11.0
5,Super Bowl LIX,2025,"New Orleans, LA",4.2,17.3,14.0
6,Super Bowl LX,2026,"Santa Clara, CA",3.4,11.2,12.0
7,Super Bowl LXI,2027,"Inglewood, CA",4.2,12.8,24.0
8,Super Bowl LXII,2028,"Atlanta, GA",3.7,11.4,8.0
9,Super Bowl LXIII,2029,To be determined,NaN,NaN,NaN


Now we have been perepared, lets do the real work:


In [515]:
#weights for the criteias of Energy consumption 
print(energy_consumption_data.columns[1:-1])
energy_consumption_matrix = np.array(energy_consumption_data[energy_consumption_data.columns[1:-1]])
energy_consumption_weights = entropy_weights(energy_consumption_matrix,[ benefit_indicator,cost_indicator,cost_indicator,benefit_indicator,benefit_indicator])
print("results for energy consumption criteria:")
energy_consumption_weights

Index(['可再生能源占比 (%)', '场馆能效 (kWh/座位)', '电网碳强度 (kgCO₂/kWh)',
       '平均太阳辐照 (kWh/m²·日)', '平均风速 (km/h)'],
      dtype='object')
results for energy consumption criteria:


array([0.34684359, 0.27562648, 0.33182509, 0.01170686, 0.03399798])

In [516]:
#weights for the criteias of emission rates 
emission_matrix = np.array(emission_data[emission_data.columns[-2:]])
# remove any row that has NaN in these two columns
emission_matrix = emission_matrix[~np.isnan(emission_matrix).any(axis=1)]
emission_weights = entropy_weights(emission_matrix,benefit_indicator)  # weights for the two criteria
print("results for emission rates:")
print("公交站数量	电车/地铁站数量")
emission_weights

results for emission rates:
公交站数量	电车/地铁站数量


array([0.21952617, 0.78047383])

In [517]:
#weights for waste management
waste_management_matrix = np.array(waste_management_data[waste_management_data.columns[-2:]])
waste_management_matrix = waste_management_matrix[~np.isnan(waste_management_matrix).any(axis=1)]

waste_management_weights = entropy_weights(waste_management_matrix, [benefit_indicator, cost_indicator])
print(waste_management_data.columns[-2:])
print("results for waste management:")
print(waste_management_weights)

Index(['分类回收率', '人均废弃物产生量 (kg/人)'], dtype='object')
results for waste management:
[0.01233383 0.98766617]


In [518]:
#weights for climate criteria
climate_matrix = np.array(climate_data[climate_data.columns[-3:]])
climate_matrix = climate_matrix[~np.isnan(climate_matrix).any(axis=1)]

climate_weights = entropy_weights(climate_matrix, [benefit_indicator,benefit_indicator,optimal_value_indicator], optimal_value = 25)
print("results for climate criteria:")
print(climate_data.columns[-3:])
climate_weights


results for climate criteria:
Index(['平均每㎡入射太阳能', '平均风速', '平均温度'], dtype='object')


array([0.01004561, 0.02404742, 0.96590697])

**===========now evaluate 1st level====================**

1st matrix---------

In [519]:
"""
             [[1. ,5. ,7.,4.   ,3],
              [1/5,1,  3,  1/2,1/3],
              [1/7,1/3,1,  1/5,1/4],
              [1/4,2. ,5,  1,  1/2],
              [1/3,3. ,4,  2,    1]] """
AHP_matric3 = np.array([     [1. ,5. ,7.,4.   ,3],
                            [1/5,1,  3,  1/3,1/5],
                            [1/7,1/3,1,  1/4,1/5],
                            [1/4,3. ,4,  1,  1/3],
                            [1/3,5. ,5,  3,    1]])




In [520]:
print('results of 3')
print(f'Normalized A:{Normalize(np.array(list(map(lambda x: np.round(x, 3), AHP_matric3))))}\n')
print('latex')
print(array_latex_table(Normalize(np.array(list(map(lambda x: np.round(x, 3), AHP_matric3))))))
print("weights")
print(AHP(AHP_matric3)[0])
print('\nCR')
print(AHP(AHP_matric3)[3])


results of 3
Normalized A:[[0.5192108  0.34884532 0.35       0.46603752 0.63384745]
 [0.10384216 0.06976906 0.15       0.03879762 0.0422565 ]
 [0.07424714 0.0232331  0.05       0.02912734 0.0422565 ]
 [0.1298027  0.20930719 0.2        0.11650938 0.07035707]
 [0.1728972  0.34884532 0.25       0.34952814 0.21128248]]

latex
0.5192107995846313 & 0.3488453219842322 & 0.35 & 0.4660375160200396 & 0.6338474540460595 \\0.10384215991692629 & 0.06976906439684644 & 0.15 & 0.0387976232086683 & 0.04225649693640397 \\0.07424714434060228 & 0.023233098444149863 & 0.05 & 0.029127344751252476 & 0.04225649693640397 \\0.12980269989615784 & 0.2093071931905393 & 0.2 & 0.1165093790050099 & 0.07035706739911261 \\0.17289719626168226 & 0.3488453219842322 & 0.25 & 0.3495281370150297 & 0.21128248468201985 \\
weights
[0.47344962 0.07476046 0.04216592 0.13971416 0.26990984]

CR
0.06777356071215065


2nd matrix------------

In [521]:
AHP_matric4 = np.array([[1, 1/7, 1/3, 1/5, 2],
                        [7, 1,   5,   3,   8],
                        [3, 1/5, 1,  1/3,  4],
                        [5, 1/3, 3,  1,    6],
                        [1/2,1/8,1/4,1/6,  1]])



In [522]:
print('results of 4')
print(f'Normalized A:{Normalize(np.array(list(map(lambda x: np.round(x, 2), AHP_matric4))))}\n')
print('latex')
print(array_latex_table(Normalize(np.array(list(map(lambda x: np.round(x, 2), AHP_matric4))))))
print("weights")
print(AHP(AHP_matric4)[0])
print('\nCR')
print(AHP(AHP_matric4)[3])

results of 4
Normalized A:[[0.06060606 0.07821229 0.03444676 0.04255319 0.0952381 ]
 [0.42424242 0.55865922 0.52192067 0.63829787 0.38095238]
 [0.18181818 0.11173184 0.10438413 0.07021277 0.19047619]
 [0.3030303  0.18435754 0.3131524  0.21276596 0.28571429]
 [0.03030303 0.06703911 0.02609603 0.03617021 0.04761905]]

latex
0.06060606060606061 & 0.0782122905027933 & 0.03444676409185804 & 0.0425531914893617 & 0.09523809523809523 \\0.42424242424242425 & 0.5586592178770949 & 0.5219206680584552 & 0.6382978723404255 & 0.38095238095238093 \\0.18181818181818182 & 0.111731843575419 & 0.10438413361169102 & 0.07021276595744681 & 0.19047619047619047 \\0.30303030303030304 & 0.18435754189944134 & 0.31315240083507306 & 0.2127659574468085 & 0.2857142857142857 \\0.030303030303030304 & 0.0670391061452514 & 0.026096033402922755 & 0.03617021276595745 & 0.047619047619047616 \\
weights
[0.05931487 0.51379444 0.12635312 0.26019561 0.04034195]

CR
0.04132174956295824
Normalized A:[[0.06060606 0.07821229 0.0344

In [523]:
def normalize_weights(weights):
    weights = np.array(weights)
    return weights / np.sum(weights)

# Compute weights for each criterion
enw = energy_consumption_weights
ew = emission_weights
ww = waste_management_weights
cw = climate_weights
water_usage_weights = np.array([1.])  # Single criterion weight
    # Combine all criteria into a single decision matrix
    
#decision_matrix = np.concatenate((energy_consumpt_dat, emission_dat, waste_mgmt_dat, climate_dat,wu))
#print(len(decision_matrix))
    # Combine weights

combined_weights = np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))* AHP(AHP_matric3)[0]))
combined_weights = normalize_weights(combined_weights)

combined_weights 

<ipython-input-523-f7c1a1c26561>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  combined_weights = np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))* AHP(AHP_matric3)[0]))


array([0.16421296, 0.13049525, 0.15710246, 0.00554261, 0.01609633,
       0.01641188, 0.05834858, 0.00052007, 0.04164585, 0.00140351,
       0.00335977, 0.13495088, 0.26990984])

In [524]:
data = pd.read_csv('/Users/daniel/Documents/Math_/Superbowl_vs_environment/data_test.csv')
print(data)
#print(len(data.columns[3:]))
data['aver'] = data['Total waste generated (ton)']/data['Number of attendees']

part0 = np.array(data[data.columns[3]].to_numpy(),dtype = float)/100
part1 = np.array(data[data.columns[4:9]].to_numpy(),dtype = float)
part2 = np.array(data['aver'].to_numpy(),dtype = float)
part3 = np.array(data['Recycling Rate'].to_numpy(),dtype = float)
part5 = np.array(data[data.columns[6:9]].to_numpy(),dtype = float)
part6 = np.array(data['Average Temperature (Celsius)'].to_numpy(),dtype = float)
part4 = np.array(data['Avalible Water Resource (tons per day)'].to_numpy(),dtype = float)

data = np.hstack((part0.reshape(-1,1),part1,part2.reshape(-1,1),part3.reshape(-1,1),part4.reshape(-1,1),part5,part6.reshape(-1,1)))
np.savetxt("/Users/daniel/Documents/Math_/Superbowl_vs_environment/data_t_test.txt", data, delimiter = ",")
topsis(data, combined_weights, ['+', '+', '+', '+', '+'])


   Super Bowl Session  Year           Cities  Renewable Energy Proportion  \
0        Super Bowl L  2016  Santa Clara, CA                          100   
1     Super Bowl LIII  2019      Atlanta, GA                          100   
2      Super Bowl LVI  2022    Inglewood, CA                           57   
3     Super Bowl LVII  2023     Glendale, AZ                           14   
4    Super Bowl LVIII  2024    Las Vegas, NV                          100   
5      Super Bowl LIX  2025  New Orleans, LA                           30   
6       Super Bowl LX  2026  Santa Clara, CA                          100   
7      Super Bowl LXI  2027    Inglewood, CA                           57   
8     Super Bowl LXII  2028      Atlanta, GA                          100   
9    Super Bowl LXIII  2029      Seattle, WA                           76   
10   Super Bowl LXIII  2029       Denver, CO                           43   
11   Super Bowl LXIII  2029    Nashville, TN                           13   

array([2.68109809e-02, 7.44318200e-01, 7.97506158e-05, 1.76933041e-01,
       9.99911217e-01, 5.31067535e-01, 2.68109809e-02, 1.45335960e-04,
       7.44318200e-01, 6.31149281e-01, 6.54244514e-01, 3.88644157e-01])

In [525]:
combined_weights = np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))* AHP(AHP_matric4)[0]))
combined_weights = np.append(combined_weights, combined_weights[-1])
combined_weights = normalize_weights(combined_weights)
combined_weights 

<ipython-input-525-922ee73f39f0>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  combined_weights = np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))* AHP(AHP_matric4)[0]))


array([0.01977521, 0.01571478, 0.01891894, 0.00066746, 0.00193839,
       0.10841755, 0.38545318, 0.00149799, 0.11995547, 0.00251246,
       0.0060144 , 0.24157899, 0.03877759, 0.03877759])

In [526]:
np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))))

<ipython-input-526-5a4ccabbbfd8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.concatenate((np.array((enw, ew, ww, cw, water_usage_weights))))


array([0.34684359, 0.27562648, 0.33182509, 0.01170686, 0.03399798,
       0.21952617, 0.78047383, 0.01233383, 0.98766617, 0.01004561,
       0.02404742, 0.96590697, 1.        ])

In [527]:
data = pd.read_csv('/Users/daniel/Documents/Math_/Superbowl_vs_environment/ababa.csv')
data = data[data.columns[2:16]]
data = data[0:5]
data['Renewable Energy Proportion'] = pd.DataFrame (map(lambda x:float(remove_unit(x,"%"))/100,data['Renewable Energy Proportion']))
data['Recycling Rate'] = pd.DataFrame (map(lambda x:float(remove_unit(x,"%") if type(x) == str else x)/100,data['Renewable Energy Proportion']))

data

,Renewable Energy Proportion,Venue Energy Efficiency (kWh/seat),Grid Carbon Intensity (kgCO_2/kWh),Daily Incident Solar Energy (energy per square meter),Average Wind Speed (km/h),Avition Carbon Emission (Mt CO_2),Avition Carbon Emission (Mt CO_2).1,Recycling Rate,Average waste generated (kg/person per day),Daily Incident Solar Energy (energy per square meter).1,Average Wind Speed (km/h).1,Average Temperature (Celsius),Avalible Water Resource (square meter/person per year),Irrigation Water for Soccer Pitch (square meter/month)
0,0.60,0.483,0.32,3.02,4.5,1.75,1.75,0.0060,1.16,3.02,4.5,22.0,1838.0,6500
1,0.08,0.380,0.87,5.50,3.5,1.65,1.65,0.0008,1.08,5.50,3.5,17.0,701.0,12000
2,0.88,0.457,0.10,5.03,4.0,2.7,2.7,0.0088,1.41,5.03,4.0,25.0,40680.0,8000
3,0.18,0.444,0.31,2.85,3.2,2.1,2.1,0.0018,1.25,2.85,3.2,24.0,31010.0,6500
4,0.00,0.405,0.49,6.02,5.0,3.6,3.6,0.0000,1.20,6.02,5.0,26.0,20.1,10000


In [528]:

topsis(data.to_numpy(dtype = float), combined_weights, ['+', '+', '+', '+', '+'])

[[6.000e-01 4.830e-01 3.200e-01 3.020e+00 4.500e+00 1.750e+00 1.750e+00
  6.000e-03 1.160e+00 3.020e+00 4.500e+00 2.200e+01 1.838e+03 6.500e+03]
 [8.000e-02 3.800e-01 8.700e-01 5.500e+00 3.500e+00 1.650e+00 1.650e+00
  8.000e-04 1.080e+00 5.500e+00 3.500e+00 1.700e+01 7.010e+02 1.200e+04]
 [8.800e-01 4.570e-01 1.000e-01 5.030e+00 4.000e+00 2.700e+00 2.700e+00
  8.800e-03 1.410e+00 5.030e+00 4.000e+00 2.500e+01 4.068e+04 8.000e+03]
 [1.800e-01 4.440e-01 3.100e-01 2.850e+00 3.200e+00 2.100e+00 2.100e+00
  1.800e-03 1.250e+00 2.850e+00 3.200e+00 2.400e+01 3.101e+04 6.500e+03]
 [0.000e+00 4.050e-01 4.900e-01 6.020e+00 5.000e+00 3.600e+00 3.600e+00
  0.000e+00 1.200e+00 6.020e+00 5.000e+00 2.600e+01 2.010e+01 1.000e+04]]
norm_matrix = [[1.09310552e-05 8.79949944e-06 5.82989611e-06 5.50196445e-05
  8.19829140e-05 3.18822444e-05 3.18822444e-05 1.09310552e-07
  2.11333734e-05 5.50196445e-05 8.19829140e-05 4.00805358e-04
  3.34854658e-02 1.18419765e-01]
 [1.45747403e-06 6.92300163e-06 1.5850030

array([0.04429407, 0.12174565, 0.91048937, 0.73584687, 0.07917986])

**问题1：理解问题 - 环境影响指标体系**
**思路**
**建立三级指标体系：**


- 一级指标：能源、交通、废弃物、水资源、气候适应性


- 二级指标：细化的可测量指标 


- 三级指标：具体数据采集点 


**关键指标** 


1. **energy consumption**


    - 可再生能源比例（%）  


    - 场馆能源效率（kWh/座位）   


    - 电网清洁度（碳强度 gCO₂/kWh）   


    - 当地太阳能/风能潜力   


1. **carbon emission by transportation**
    - 平均观众出行距离（km）   


    - 公共交通覆盖率（%）   


    - 机场碳排放效率   


    - 电动车充电基础设施密度   


1. **waste management**
    - 回收率（%）   


    - 人均废弃物产生量（kg/人）   


    - 堆肥设施可用性   


    - 一次性塑料使用政策   


1. **water usage**
    - 水资源压力指数   


    - 雨水回收系统   


    - 节水技术应用率   


1. **climate**


    - 热岛效应强度   